In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
# =============================================================================
# 1. IMPORTS & SETUP
# =============================================================================
import re
import os
import logging
import time # <-- ADDED: For time tracking
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

# =============================================================================
# 2. CONFIGURATION
# =============================================================================
class TrainingConfig:
    MODEL_NAME = "law-ai/InLegalBERT"
    EPOCHS = 5
    BATCH_SIZE = 32
    MAX_LEN = 256
    LEARNING_RATE = 2e-5
    N_SPLITS = 5
    RANDOM_STATE = 42
    TEST_SET_SIZE = 0.2
    DROPOUT_1 = 0.1
    DROPOUT_2 = 0.4
    OUTPUT_DIR = "model_checkpoints"
    LOG_FILE = "training_and_testing.log"
    RESUME_CHECKPOINT_FILE = "resume_checkpoint.pt"
    POS_DATA_FILE = "/kaggle/input/lrec-dataset/sentencePair.txt"
    NEG_DATA_FILE = "/kaggle/input/lrec-dataset/sentencePair_neg.txt"
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============================================================================
# 3. DATA LOADING & PREPARATION
# =============================================================================
def parse_lrec_line(line: str):
    # (This function remains unchanged)
    parts = line.strip().split("\t")
    fname_indices = [i for i, p in enumerate(parts) if p.endswith(".txt")]
    if len(fname_indices) != 2: return None
    try:
        sentpair_id = int(parts[0])
        sent1 = " ".join(parts[fname_indices[0] + 2 : fname_indices[1]]).strip('"')
        sent2 = " ".join(parts[fname_indices[1] + 2 : len(parts) - 2]).strip('"')
        label = parts[-1]
    except (ValueError, IndexError): return None
    return {"id": sentpair_id, "sent1": sent1, "sent2": sent2, "label": label}

def load_and_split_data(config: TrainingConfig):
    # (This function remains unchanged)
    logging.info("Loading and preparing data...")
    rows = []
    for filepath in [config.POS_DATA_FILE, config.NEG_DATA_FILE]:
        if not os.path.exists(filepath):
            logging.error(f"Data file not found at '{filepath}'.")
            return None, None
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                if parsed := parse_lrec_line(line): rows.append(parsed)
    df = pd.DataFrame(rows)
    label_map = {"SUPPORT": 0, "ATTACK": 1, "NO_REL": 2}
    df["label_id"] = df["label"].map(label_map)
    df = df.dropna(subset=['id', 'sent1', 'sent2', 'label_id', 'label'])
    train_df, test_df = train_test_split(df, test_size=config.TEST_SET_SIZE, random_state=config.RANDOM_STATE, stratify=df['label_id'])
    logging.info(f"Data loaded. Training samples: {len(train_df)}, Test samples: {len(test_df)}")
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

# =============================================================================
# 4. PYTORCH DATASET & MODEL (Unchanged)
# =============================================================================
class LegalDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df, self.tokenizer, self.max_len = df, tokenizer, max_len
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sent1, sent2, label = str(row["sent1"]), str(row["sent2"]), row["label_id"]
        encoded = self.tokenizer(sent1, sent2, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt")
        return {"input_ids": encoded["input_ids"].squeeze(), "attention_mask": encoded["attention_mask"].squeeze(), "labels": torch.tensor(label, dtype=torch.long)}

class BertSentencePairClassifier(nn.Module):
    def __init__(self, config: TrainingConfig):
        super(BertSentencePairClassifier, self).__init__()
        self.bert, self.dropout1 = AutoModel.from_pretrained(config.MODEL_NAME, trust_remote_code=True), nn.Dropout(config.DROPOUT_1)
        self.hidden, self.relu = nn.Linear(768, 384), nn.ReLU()
        self.dropout2, self.out = nn.Dropout(config.DROPOUT_2), nn.Linear(384, 3)
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        x = self.dropout1(cls_embedding); x = self.hidden(x); x = self.relu(x); x = self.dropout2(x); logits = self.out(x)
        return logits

# =============================================================================
# 6. TRAINING & EVALUATION PIPELINE (WITH TIME LOGGING)
# =============================================================================
class Trainer:
    def __init__(self, config: TrainingConfig, train_df: pd.DataFrame, logger):
        self.config, self.train_df, self.logger = config, train_df, logger
        self.tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME, trust_remote_code=True)
        os.makedirs(config.OUTPUT_DIR, exist_ok=True)

    def _save_resume_checkpoint(self, fold, epoch, model, optimizer, scheduler):
        # (This method remains unchanged)
        checkpoint_path = os.path.join(self.config.OUTPUT_DIR, self.config.RESUME_CHECKPOINT_FILE)
        state = {'fold': fold, 'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'scheduler_state_dict': scheduler.state_dict()}
        torch.save(state, checkpoint_path)
        self.logger.info(f"Resume checkpoint saved for Fold {fold+1}, Epoch {epoch+1}")

    def _load_resume_checkpoint(self):
        # (This method remains unchanged)
        checkpoint_path = os.path.join(self.config.OUTPUT_DIR, self.config.RESUME_CHECKPOINT_FILE)
        if os.path.exists(checkpoint_path):
            self.logger.info(f"Found resume checkpoint at {checkpoint_path}. Loading state.")
            return torch.load(checkpoint_path)
        return None

    def run_kfold_cv(self):
        checkpoint = self._load_resume_checkpoint()
        start_fold = checkpoint['fold'] if checkpoint else 0
        kf = KFold(n_splits=self.config.N_SPLITS, shuffle=True, random_state=self.config.RANDOM_STATE)
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(self.train_df)):
            if fold < start_fold: continue
            
            fold_start_time = time.time() # <-- TIME TRACKING FOR FOLD
            self.logger.info(f"\n{'='*20} FOLD {fold+1}/{self.config.N_SPLITS} {'='*20}")
            train_sub_df, val_df = self.train_df.iloc[train_idx], self.train_df.iloc[val_idx]
            train_ds, val_ds = LegalDataset(train_sub_df, self.tokenizer, self.config.MAX_LEN), LegalDataset(val_df, self.tokenizer, self.config.MAX_LEN)
            train_dl, val_dl = DataLoader(train_ds, batch_size=self.config.BATCH_SIZE, shuffle=True), DataLoader(val_ds, batch_size=self.config.BATCH_SIZE)
            # Create the model first
            model = BertSentencePairClassifier(self.config).to(self.config.DEVICE)
            # Then create the optimizer using the model
            optimizer = AdamW(model.parameters(), lr=self.config.LEARNING_RATE)
            scheduler, criterion = get_linear_schedule_with_warmup(optimizer, 0, len(train_dl) * self.config.EPOCHS), nn.CrossEntropyLoss()
            
            start_epoch = 0
            if checkpoint and checkpoint['fold'] == fold:
                self.logger.info("Resuming from a saved state within this fold.")
                model.load_state_dict(checkpoint['model_state_dict']); optimizer.load_state_dict(checkpoint['optimizer_state_dict']); scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
            
            for epoch in range(start_epoch, self.config.EPOCHS):
                epoch_start_time = time.time() # <-- TIME TRACKING FOR EPOCH
                train_loss, train_acc = self.train_epoch(model, train_dl, optimizer, scheduler, criterion)
                val_loss, val_acc, precision, recall, f1 = self.eval_metrics(model, val_dl, criterion)
                
                epoch_end_time = time.time()
                self.logger.info(f"--- Epoch {epoch+1}/{self.config.EPOCHS} (Duration: {epoch_end_time - epoch_start_time:.2f} seconds) ---")
                self.logger.info(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
                self.logger.info(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1:.4f}")
                
                self._save_resume_checkpoint(fold, epoch, model, optimizer, scheduler)
                for handler in self.logger.handlers: handler.flush()

            model_save_path = os.path.join(self.config.OUTPUT_DIR, f"model_fold_{fold+1}")
            os.makedirs(model_save_path, exist_ok=True)
            model.bert.save_pretrained(model_save_path)
            torch.save(model.state_dict(), os.path.join(model_save_path, 'pytorch_model.bin'))
            self.tokenizer.save_pretrained(model_save_path)
            
            fold_end_time = time.time()
            self.logger.info(f" Final model for fold {fold+1} saved. (Fold duration: {fold_end_time - fold_start_time:.2f} seconds)")

        checkpoint_path = os.path.join(self.config.OUTPUT_DIR, self.config.RESUME_CHECKPOINT_FILE)
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)
            self.logger.info("Training complete. Removed resume checkpoint file.")

    def evaluate_on_test_set(self, test_df, model_load_location=None):
        self.logger.info(f"\n{'='*25}\nEVALUATING ON TEST SET\n{'='*25}")
        test_start_time = time.time() # <-- TIME TRACKING FOR TEST
        # ... (rest of the method is the same, with added logging at the end)
        base_path = model_load_location if model_load_location is not None else self.config.OUTPUT_DIR
        model_paths_to_test = []
        if os.path.exists(os.path.join(base_path, 'pytorch_model.bin')):
            model_paths_to_test.append(base_path)
        else:
            for item in os.listdir(base_path):
                full_path = os.path.join(base_path, item)
                if os.path.isdir(full_path) and item.startswith("model_fold_"): model_paths_to_test.append(full_path)
        
        if not model_paths_to_test:
            self.logger.error(f"No models found at the specified location: {base_path}")
            return

        test_dl = DataLoader(LegalDataset(test_df, self.tokenizer, self.config.MAX_LEN), batch_size=self.config.BATCH_SIZE)
        id_to_label = {v: k for k, v in {"SUPPORT": 0, "ATTACK": 1, "NO_REL": 2}.items()}

        for model_path in model_paths_to_test:
            fold_name = os.path.basename(model_path)
            self.logger.info(f"\n--- Loading and testing model: {fold_name} ---")
            model = BertSentencePairClassifier(self.config).to(self.config.DEVICE)
            model.load_state_dict(torch.load(os.path.join(model_path, 'pytorch_model.bin')))
            model.eval()
            all_preds, all_labels = [], []
            with torch.no_grad():
                for batch in tqdm(test_dl, desc=f"Testing {fold_name}"):
                    outputs = model(batch["input_ids"].to(self.config.DEVICE), batch["attention_mask"].to(self.config.DEVICE))
                    all_preds.extend(outputs.argmax(dim=1).cpu().numpy()); all_labels.extend(batch["labels"].cpu().numpy())
            
            self.logger.info(f"Test Results for {fold_name}:")
            report = classification_report(all_labels, all_preds, target_names=id_to_label.values(), zero_division=0)
            self.logger.info("Classification Report:\n" + report)
            
            output_df = pd.DataFrame({'pair_id': test_df['id'], 'sentence1': test_df['sent1'], 'sentence2': test_df['sent2'], 'original_label': test_df['label'], 'prediction': [id_to_label[pred] for pred in all_preds]})
            csv_path = os.path.join(self.config.OUTPUT_DIR, f"test_predictions_{fold_name}.csv")
            output_df.to_csv(csv_path, index=False)
            self.logger.info(f" Test predictions for {fold_name} saved to {csv_path}")

        test_end_time = time.time()
        self.logger.info(f"\nTotal testing duration: {test_end_time - test_start_time:.2f} seconds")

    # (train_epoch and eval_metrics are unchanged)
    def train_epoch(self, model, dataloader, optimizer, scheduler, criterion):
        model.train(); total_loss, total_correct = 0, 0
        for batch in tqdm(dataloader, desc="Training"):
            optimizer.zero_grad()
            outputs = model(batch["input_ids"].to(self.config.DEVICE), batch["attention_mask"].to(self.config.DEVICE))
            loss = criterion(outputs, batch["labels"].to(self.config.DEVICE))
            loss.backward(); optimizer.step(); scheduler.step()
            total_loss += loss.item(); total_correct += (outputs.argmax(dim=1) == batch["labels"].to(self.config.DEVICE)).sum().item()
        return total_loss / len(dataloader), total_correct / len(dataloader.dataset)

    def eval_metrics(self, model, dataloader, criterion):
        model.eval(); total_loss, all_preds, all_labels = 0, [], []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Evaluating"):
                outputs = model(batch["input_ids"].to(self.config.DEVICE), batch["attention_mask"].to(self.config.DEVICE))
                loss = criterion(outputs, batch["labels"].to(self.config.DEVICE))
                total_loss += loss.item(); all_preds.extend(outputs.argmax(dim=1).cpu().numpy()); all_labels.extend(batch["labels"].cpu().numpy())
        acc = accuracy_score(all_labels, all_preds); precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted", zero_division=0)
        return total_loss / len(dataloader), acc, precision, recall, f1

# =============================================================================
# 7. MAIN EXECUTION BLOCK
# =============================================================================
if __name__ == '__main__':
    config = TrainingConfig()
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler, stream_handler = logging.FileHandler(config.LOG_FILE), logging.StreamHandler()
    file_handler.setFormatter(formatter); stream_handler.setFormatter(formatter)
    if not logger.handlers: # Avoid adding handlers multiple times in notebooks
        logger.addHandler(file_handler); logger.addHandler(stream_handler)

    logger.info(f"Using device: {config.DEVICE}")
    train_df, test_df = load_and_split_data(config)
    if train_df is not None and test_df is not None:
        trainer = Trainer(config, train_df, logger)
        logger.info("Starting training process...")
        trainer.run_kfold_cv()
        logger.info("Training complete. Starting evaluation on the test set...")
        trainer.evaluate_on_test_set(test_df)
    else:
        logger.error("Halting execution due to data loading errors.")

2025-10-14 08:40:06,763 - INFO - Using device: cuda


tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

2025-10-14 08:40:08,047 - INFO - Starting training process...
2025-10-14 08:40:08,050 - INFO - 
==================== FOLD 1/5 ====================


...

2025-10-14 17:14:01,441 - INFO - Test Results for model_fold_4:
2025-10-14 17:14:01,456 - INFO - Classification Report:
              precision    recall  f1-score   support

     SUPPORT       0.79      0.84      0.81      2150
      ATTACK       0.77      0.80      0.79      1952
      NO_REL       0.93      0.89      0.91      4000

    accuracy                           0.85      8102
   macro avg       0.83      0.84      0.84      8102
weighted avg       0.86      0.85      0.86      8102

2025-10-14 17:14:01,590 - INFO -  Test predictions for model_fold_4 saved to model_checkpoints/test_predictions_model_fold_4.csv
2025-10-14 17:14:01,591 - INFO - 
--- Loading and testing model: model_fold_2 ---


Testing model_fold_2:   0%|          | 0/254 [00:00<?, ?it/s]

2025-10-14 17:16:02,780 - INFO - Test Results for model_fold_2:
2025-10-14 17:16:02,795 - INFO - Classification Report:
              precision    recall  f1-score   support

     SUPPORT       0.80      0.81      0.81      2150
      ATTACK       0.76      0.81      0.78      1952
      NO_REL       0.93      0.90      0.92      4000

    accuracy                           0.85      8102
   macro avg       0.83      0.84      0.84      8102
weighted avg       0.86      0.85      0.85      8102

2025-10-14 17:16:02,940 - INFO -  Test predictions for model_fold_2 saved to model_checkpoints/test_predictions_model_fold_2.csv
2025-10-14 17:16:02,941 - INFO - 
--- Loading and testing model: model_fold_1 ---


Testing model_fold_1:   0%|          | 0/254 [00:00<?, ?it/s]

2025-10-14 17:18:06,049 - INFO - Test Results for model_fold_1:
2025-10-14 17:18:06,063 - INFO - Classification Report:
              precision    recall  f1-score   support

     SUPPORT       0.81      0.81      0.81      2150
      ATTACK       0.77      0.80      0.78      1952
      NO_REL       0.93      0.91      0.92      4000

    accuracy                           0.86      8102
   macro avg       0.83      0.84      0.84      8102
weighted avg       0.86      0.86      0.86      8102

2025-10-14 17:18:06,197 - INFO -  Test predictions for model_fold_1 saved to model_checkpoints/test_predictions_model_fold_1.csv
2025-10-14 17:18:06,198 - INFO - 
Total testing duration: 609.46 seconds
